<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD Tutorial

FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the [CeRAS reference case](http://ceras.ilr.rwth-aachen.de/trac/wiki/CeRAS/AircraftDesigns/CSR01). In `workdir/`, we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import openmdao.api as om
import logging
import shutil
import fastoad.api as oad


DATA_FOLDER_PATH = 'data'

WORK_FOLDER_PATH = 'workdir'

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'oad_process.yml')
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, 'CeRAS01_baseline.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


After defining a configuration file name, we can ask FAST-OAD to generate a default configuration file based on the default OAD model implemented in the framework:

In [2]:
oad.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

# The above generated configuration uses the quick and simple Breguet module to compute performances. 
# If you want to use the more accurate, much more CPU-costly time-step integration, uncomment and run 
# the next line:
#shutil.copy(pth.join(DATA_FOLDER_PATH, "oad_process_timestep_mission.yml"), CONFIGURATION_FILE)

INFO    : Sample configuration written in workdir\oad_process.yml


You can now checkout the generated [configuration file](./workdir/oad_process.yml).

In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [3]:
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Problem inputs written in C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\notebooks\01_tutorial\workdir\problem_inputs.xml


'C:\\Users\\Mathieu\\PycharmProjects\\FAST-OAD\\src\\fastoad\\notebooks\\01_tutorial\\workdir\\problem_inputs.xml'

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [4]:
oad.list_variables(CONFIGURATION_FILE)

NAME,I/O,DESCRIPTION
data:TLAR:NPAX,IN,"top-level requirement: number of passengers, assuming a classic eco/business class repartition"
data:TLAR:approach_speed,IN,top-level requirement: approach speed
data:TLAR:cruise_mach,IN,Input defined by the mission.
data:TLAR:range,IN,Input defined by the mission.
data:aerodynamics:aircraft:landing:CL_max_clean_2D,IN,maximum lift coefficient of 2D average profile in landing conditions without high-lift devices
data:aerodynamics:aircraft:takeoff:mach,IN,considered Mach number for takeoff phase
data:geometry:cabin:aisle_width,IN,width of aisles
data:geometry:cabin:containers:count_by_row,IN,number of cargo containers along width
data:geometry:cabin:crew_count:technical,IN,number of technical crew members
data:geometry:cabin:exit_width,IN,width of exits


Another useful feature is to list the modules of the model defined in the configuration file:

In [5]:
oad.list_modules(CONFIGURATION_FILE)
#api.list_modules(CONFIGURATION_FILE, verbose=True) # Use this line instead of the previous one for detailed information (text format only) 

AVAILABLE MODULE IDENTIFIERS,MODULE PATH
fastoad.aerodynamics.highspeed.legacy,C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\models\aerodynamics\aerodynamics_high_speed.py
fastoad.aerodynamics.landing.legacy,C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\models\aerodynamics\aerodynamics_landing.py
fastoad.aerodynamics.lowspeed.legacy,C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\models\aerodynamics\aerodynamics_low_speed.py
fastoad.aerodynamics.takeoff.legacy,C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\models\aerodynamics\aerodynamics_takeoff.py
fastoad.geometry.legacy,C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\models\geometry\geometry.py
fastoad.handling_qualities.static_margin,C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\models\handling_qualities\compute_static_margin.py
fastoad.handling_qualities.tail_sizing,C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\models\handling_qualities\tail_sizing\compute_tail_areas.py
fastoad.loop.wing_area,C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\models\loops\compute_wing_area.py
fastoad.loop.wing_position,C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\models\loops\compute_wing_position.py
fastoad.mass_performances.compute_MTOW,C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\models\performances\mission\openmdao\link_mtow.py


Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [6]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

INFO    : N2 diagram written in C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\notebooks\01_tutorial\workdir\n2.html


Alternatively, you can create a [WhatsOpt](https://github.com/OneraHub/WhatsOpt-Doc#whatsopt-documentation) account to generate the XDSM of the problem. If your account is created, you may uncomment next lines and run them (this should take ~ 1 min):

In [7]:
# XDSM_FILE = pth.join(WORK_FOLDER_PATH, 'xdsm.html')
# oad.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
# from IPython.display import IFrame
# IFrame(src=XDSM_FILE, width='100%', height='500px')

## 2. Running your first MDA

### CeRAS
Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

*(This run should take around 10 seconds)*

In [8]:
eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Mission computation - iteration 0 : Using initializer computation. OTHER ITERATIONS NEEDED.
INFO    : Mission computation - iteration 1 : Using mission definition.
INFO    : Mission computation - iteration 2 : Using mission definition.
INFO    : Mission computation - iteration 3 : Using mission definition.
INFO    : Mission computation - iteration 4 : Using mission definition.
INFO    : Mission computation - iteration 5 : Using mission definition.
INFO    : Mission computation - iteration 6 : Using mission definition.
INFO    : Mission computation - iteration 7 : Using mission definition.
INFO    : Mission computation - iteration 8 : Using mission definition.
INFO    : Mission computation - iteration 9 : Using mission definition.
INFO    : Mission computation - iteration 10 : Using mission definition.
INFO    : Mission computation - iteration 11 : Using mission definition.
INFO    : Mission computation - iteration 12 : Using mission definition.
INFO    : Mission computation -

NL: NLBGS Converged in 35 iterations


INFO    : Computation finished after 18.41 seconds
INFO    : Problem outputs written in C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\notebooks\01_tutorial\workdir\problem_outputs.xml


> _**Note**: the warning you see at end of computation is normal. FAST-OAD allows to define validity domain for each model, and the propulsion model has been validated for a certain range of turbine inlet temperature (a.k.a. T4). The T4 value we use is slightly outside the validated range, hence the warning. The check of validity domain issues warnings as soon as a value is outside the bounds, but in the end, it's up to the user to decide if the problem is real or not, though it may require a certain knowledge about the used models.  
> In this case, the model has been validated for T4 between 1400K and 1600K and using it for a value of 1633K can be considered as OK._

Let's save these results. We will use them in the next notebook, that shows some post-processing utilities.

In [9]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_CeRAS_2750nm_mda.xml')
shutil.copy(OUTPUT_FILE, CeRAS_OUTPUT_FILE)
MISSION_FILE = pth.join(WORK_FOLDER_PATH, 'flight_points.csv')
CeRAS_MISSION_FILE = pth.join(WORK_FOLDER_PATH, 'mission_CeRAS_2750nm_mda.csv')
shutil.copy(MISSION_FILE, CeRAS_MISSION_FILE)

'workdir\\mission_CeRAS_2750nm_mda.csv'

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [10]:
oad.variable_viewer(OUTPUT_FILE)

### CeRAS for 800 nm range
Here we run an MDA but we change one of the Top Level Aircraft Requirement (TLAR): the range. We choose a 800 nm range instead of the 2750 nm of the CeRAS. For that we use the `VariableViewer` tool on the input file to change the range (do not forget to save!). Just like this:
![variable_viewer](./img/variable_viewer_change_range.gif)

In [11]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs.xml')
oad.variable_viewer(INPUT_FILE)

You can also do this operation through Python like this:

In [12]:
input_data = oad.DataFile(INPUT_FILE)
input_data["data:TLAR:range"].value = 800.0
input_data.save()

Now that the range has been changed, we run again the MDA.

*(This run should take around 10 seconds)*

In [13]:
eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Mission computation - iteration 0 : Using initializer computation. OTHER ITERATIONS NEEDED.
INFO    : Mission computation - iteration 1 : Using mission definition.
INFO    : Mission computation - iteration 2 : Using mission definition.
INFO    : Mission computation - iteration 3 : Using mission definition.
INFO    : Mission computation - iteration 4 : Using mission definition.
INFO    : Mission computation - iteration 5 : Using mission definition.
INFO    : Mission computation - iteration 6 : Using mission definition.
INFO    : Mission computation - iteration 7 : Using mission definition.
INFO    : Mission computation - iteration 8 : Using mission definition.
INFO    : Mission computation - iteration 9 : Using mission definition.
INFO    : Mission computation - iteration 10 : Using mission definition.
INFO    : Mission computation - iteration 11 : Using mission definition.
INFO    : Mission computation - iteration 12 : Using mission definition.
INFO    : Mission computation -

NL: NLBGS Converged in 30 iterations


INFO    : Computation finished after 15.27 seconds
INFO    : Problem outputs written in C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\notebooks\01_tutorial\workdir\problem_outputs.xml


Let's save again these new results, for post-processing them in next notebook.

In [14]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_800nm_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_CeRAS_800nm_mda.xml')
shutil.copy(OUTPUT_FILE, CeRAS_800nm_OUTPUT_FILE)
MISSION_FILE = pth.join(WORK_FOLDER_PATH, 'flight_points.csv')
CeRAS_800nm_MISSION_FILE = pth.join(WORK_FOLDER_PATH, 'mission_CeRAS_800nm_mda.csv')
shutil.copy(MISSION_FILE, CeRAS_800nm_MISSION_FILE)

'workdir\\mission_CeRAS_800nm_mda.csv'

## 3. Running your first MDO

## CeRAS 2750 nm optimization

It is possible to run an optimization problem on top of your analysis. Thus, you can choose to set some global inputs as design variables, and global outputs as constraints or the objective. Here is an example given in the default configuration file that aims at:
- minimizing the fuel consumption for the mission (objective),
- with respect to the wing aspect ratio (design variables),
- subject to a wing span limit (constraints).

The last part of the configuration file .yaml is where this optimization problem is defined:

```yaml
optimization:  # This section is needed only if optimization process is run
  design_variables:
    - name: data:geometry:wing:aspect_ratio
      lower: 9.0
      upper: 18.0
  constraints:
    - name: data:geometry:wing:span
      upper: 60.0
  objective:
    - name: data:mission:sizing:needed_block_fuel
      scaler: 1.e-4
```

Note that we need to use [scaling](https://openmdao.org/twodocs/versions/latest/features/core_features/defining_components/scaling.html) on the objective (fuel mass for the mission) so that it has the same order of magnitude than the design variable (wing aspect ratio) by adding the line `scaler = 1e-4`. Without this the optimizer might not detect that the steps it makes on the wing aspect ratio has an effect on decreasing the fuel mass and could result in an early stop.


*(This run should take around 3 minutes)*

In [15]:
# Set back the inputs from the reference CeRAS 2750 nm
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Problem inputs written in C:\Users\Mathieu\PycharmProjects\FAST-OAD\src\fastoad\notebooks\01_tutorial\workdir\problem_inputs.xml


'C:\\Users\\Mathieu\\PycharmProjects\\FAST-OAD\\src\\fastoad\\notebooks\\01_tutorial\\workdir\\problem_inputs.xml'

To visualize and edit the optimization problem definition (present in the configuration file .yml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .yml whereas `Value` corresponds to the value found in the output file defined in the configuration file (here it is the 800 nm MDA run).

In [16]:
oad.optimization_viewer(CONFIGURATION_FILE)

In [17]:
optim_problem = oad.optimize_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Mission computation - iteration 0 : Using initializer computation. OTHER ITERATIONS NEEDED.
INFO    : Mission computation - iteration 1 : Using mission definition.
INFO    : Mission computation - iteration 2 : Using mission definition.
INFO    : Mission computation - iteration 3 : Using mission definition.
INFO    : Mission computation - iteration 4 : Using mission definition.
INFO    : Mission computation - iteration 5 : Using mission definition.
INFO    : Mission computation - iteration 6 : Using mission definition.
INFO    : Mission computation - iteration 7 : Using mission definition.
INFO    : Mission computation - iteration 8 : Using mission definition.
INFO    : Mission computation - iteration 9 : Using mission definition.
INFO    : Mission computation - iteration 10 : Using mission definition.
INFO    : Mission computation - iteration 11 : Using mission definition.
INFO    : Mission computation - iteration 12 : Using mission definition.
INFO    : Mission computation -

NL: NLBGS Converged in 35 iterations


INFO    : Mission computation - iteration 35 : Using mission definition.
INFO    : Mission computation - iteration 36 : Using mission definition.


NL: NLBGS Converged in 2 iterations


INFO    : Mission computation - iteration 37 : Using mission definition.
INFO    : Mission computation - iteration 38 : Using mission definition.
INFO    : Mission computation - iteration 39 : Using mission definition.
INFO    : Mission computation - iteration 40 : Using mission definition.
INFO    : Mission computation - iteration 41 : Using mission definition.
INFO    : Mission computation - iteration 42 : Using mission definition.
INFO    : Mission computation - iteration 43 : Using mission definition.
INFO    : Mission computation - iteration 44 : Using mission definition.
INFO    : Mission computation - iteration 45 : Using mission definition.
INFO    : Mission computation - iteration 46 : Using mission definition.
INFO    : Mission computation - iteration 47 : Using mission definition.
INFO    : Mission computation - iteration 48 : Using mission definition.
INFO    : Mission computation - iteration 49 : Using mission definition.
INFO    : Mission computation - iteration 50 : Usin

...

NL: NLBGS Converged in 24 iterations


INFO    : Mission computation - iteration 399 : Using mission definition.
INFO    : Mission computation - iteration 400 : Using mission definition.
INFO    : Mission computation - iteration 401 : Using mission definition.
INFO    : Mission computation - iteration 402 : Using mission definition.
INFO    : Mission computation - iteration 403 : Using mission definition.
INFO    : Mission computation - iteration 404 : Using mission definition.
INFO    : Mission computation - iteration 405 : Using mission definition.
INFO    : Mission computation - iteration 406 : Using mission definition.
INFO    : Mission computation - iteration 407 : Using mission definition.
INFO    : Mission computation - iteration 408 : Using mission definition.
INFO    : Mission computation - iteration 409 : Using mission definition.
INFO    : Mission computation - iteration 410 : Using mission definition.
INFO    : Mission computation - iteration 411 : Using mission definition.
INFO    : Mission computation - iterat

NL: NLBGS Converged in 25 iterations


INFO    : Mission computation - iteration 424 : Using mission definition.
INFO    : Mission computation - iteration 425 : Using mission definition.
INFO    : Mission computation - iteration 426 : Using mission definition.
INFO    : Mission computation - iteration 427 : Using mission definition.
INFO    : Mission computation - iteration 428 : Using mission definition.
INFO    : Mission computation - iteration 429 : Using mission definition.
INFO    : Mission computation - iteration 430 : Using mission definition.
INFO    : Mission computation - iteration 431 : Using mission definition.
INFO    : Mission computation - iteration 432 : Using mission definition.
INFO    : Mission computation - iteration 433 : Using mission definition.
INFO    : Mission computation - iteration 434 : Using mission definition.
INFO    : Mission computation - iteration 435 : Using mission definition.
INFO    : Mission computation - iteration 436 : Using mission definition.
INFO    : Mission computation - iterat

FileNotFoundError: 'performance.mission_computation' <class MissionComponent>: Error calling compute(), [Errno 2] No such file or directory: 'C:\\Users\\Mathieu\\PycharmProjects\\FAST-OAD\\src\\fastoad\\notebooks\\01_tutorial\\workdir\\./flight_points.csv'

Let's save these results:

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_OPT_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_CeRAS_2750nm_mdo.xml')
shutil.copy(OUTPUT_FILE, CeRAS_OPT_OUTPUT_FILE)
MISSION_FILE = pth.join(WORK_FOLDER_PATH, 'flight_points.csv')
CeRAS_OPT_MISSION_FILE = pth.join(WORK_FOLDER_PATH, 'mission_CeRAS_2750nm_mdo.csv')
shutil.copy(MISSION_FILE, CeRAS_OPT_MISSION_FILE)

The `optimizer_viewer` offers a convenient summary of the optimization result:

In [ ]:
oad.optimization_viewer(CONFIGURATION_FILE)

We see that the optimizer finds a compromise between a smaller induced drag (higher aspect ratio) and a heavier wing.

**Note: Finding such compromise on real aircraft is still a subject of research. Please take this result as informational.**

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [ ]:
RESULT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')

In [ ]:
oad.variable_viewer(RESULT_FILE)